# Introduction to NGSILDCLIENT

* Instance the connection to context broker from nsildclient

In [1]:
from ngsildclient import Client, Entity, iso8601

client_happy = Client(hostname="52.233.87.128",port=1029)
client_real = Client(hostname="52.233.87.128",port=1026)
client_smart = Client(hostname="52.233.87.128",port=1028)


Connected to Context Broker at 52.233.87.128:1029 | tenant=N/A | vendor=Orion-LD | version=1.0.1-PRE-468

Connected to Context Broker at 52.233.87.128:1026 | tenant=N/A | vendor=Orion-LD | version=1.0.1-PRE-468

Connected to Context Broker at 52.233.87.128:1028 | tenant=N/A | vendor=Orion-LD | version=1.0.1-PRE-468

* Create suscriptions

In [13]:
from ngsildclient import SubscriptionBuilder

subscr = SubscriptionBuilder("http://ai.notification-proxy.docker:8080/notification")\
    .description("Notify me of new animal coordinates")\
    .select_type("Animal")\
    .watch(["location"])\
    .notif(["location"])\
    .build()
client_happy.subscriptions.create(subscr)

'urn:ngsi-ld:Subscription:66301de7b6db47c3ce445c19'

In [14]:

subscr = SubscriptionBuilder("http://apis.docker:5000/notification")\
    .description("Notify AI service of new animal coordinates")\
    .select_type("Animal")\
    .watch(["location"])\
    .notif(["location"])\
    .build()
client_smart.subscriptions.create(subscr)

'urn:ngsi-ld:Subscription:66301de92255993c327ba963'

In [15]:

subscr = SubscriptionBuilder("http://farm.notification-proxy.docker:8081/notification")\
    .description("Notify me of new animal activity prediction")\
    .select_type("Animal")\
    .watch(["animalActivity"])\
    .notif(["animalActivity"])\
    .build()
client_smart.subscriptions.create(subscr)

'urn:ngsi-ld:Subscription:66301dea2255993c327ba964'

* Create a temperature sensor entity from function def.

In [16]:
def build_entity(sensor: dict) -> Entity:
    e = Entity("TemperatureSensor", f"{sensor['id']}")
    e.obs()
    e.prop("temperature", sensor["temperature"])
    return e


In [3]:
def build_entity_animal(animal: dict) -> Entity:
    e = Entity("Animal", f"{animal['id']}")
    e.obs()
    e.prop("species", animal["species"])
    e.gprop("location", animal['location']) 
    return e

In [18]:
tempSensor = {"id": "001", "temperature": 17}
entity = build_entity(tempSensor)
print (entity)
#upser entity in client
client_real.upsert(entity)

{'id': 'urn:ngsi-ld:TemperatureSensor:001', 'type': 'TemperatureSensor', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2024-04-29T22:21:17Z'}}, 'temperature': {'type': 'Property', 'value': 17}}


True

In [4]:
from geojson import Point

animal = {"id": "0002", "species": "cow", "location": Point((3.163,5.6133,-1.4292))}
entity = build_entity_animal(animal)
print (entity)
#upser entity in client
client_happy.upsert(entity)

{'id': 'urn:ngsi-ld:Animal:0002', 'type': 'Animal', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2024-04-29T22:25:28Z'}}, 'species': {'type': 'Property', 'value': 'cow'}, 'location': {'type': 'GeoProperty', 'value': {"coordinates": [3.163, 5.6133, -1.4292], "type": "Point"}}}


True

* GET entities of certain type.

In [20]:
client_real.query(type='TemperatureSensor')

[{'@context': 'https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld', 'id': 'urn:ngsi-ld:TemperatureSensor:001', 'type': 'TemperatureSensor', 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2024-04-29T22:21:17Z'}}, 'temperature': {'type': 'Property', 'value': 17}}]

* GET entity given urn.

In [5]:
client_smart.get('urn:ngsi-ld:Animal:0002')

{'@context': 'https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld', 'id': 'urn:ngsi-ld:Animal:0002', 'type': 'Animal', 'location': {'type': 'GeoProperty', 'value': {'type': 'Point', 'coordinates': [3.163, 5.6133, -1.4292]}}, 'animalActivity': {'value': 'Sternal lying', 'type': 'Property'}}